In [20]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F

from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

In [21]:
iris = datasets.load_iris()
X = iris.data # Features
y = iris.target # Labels

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Initalize the K and Train the KNN Classifer
k = 5
knn_clf = KNeighborsClassifier(n_neighbors=k)
knn_clf.fit(X_train, y_train)

# make perdiction
y_pred = knn_clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy (Sklearn): {accuracy:.2f}")
print(f"Classificatoin Report:", classification_report(y_test, y_pred))


Accuracy (Sklearn): 1.00
Classificatoin Report:               precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00         9
           2       1.00      1.00      1.00        11

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30



In [22]:
# Convert into tensors
X_train_tensors = torch.tensor(X_train, dtype=torch.float32)
X_test_tensors = torch.tensor(X_test, dtype=torch.float32)
y_train_tensors = torch.tensor(y_train, dtype=torch.long)
y_test_tensors = torch.tensor(y_test, dtype=torch.long)

# Implementaiton of KNN

class KNNClassifier():
    def __init__(self, k=5):
        self.k = k
        
    def fit(self, X, y):
        self.X_train = X
        self.y_train = y
    
    def predict(self, X):
        distance = torch.cdist(X, self.X_train)
        knn_indices = distance.topk(self.k, largest=False).indices
        knn_labels = self.y_train[knn_indices]
        y_pred = torch.mode(knn_labels, dim=1).values
        return y_pred
    
# Train & Test the Py-torch KNN
knn = KNNClassifier(5)
knn.fit(X_train_tensors, y_train_tensors)
y_pred_torch = knn.predict(X_test_tensors)

# Evaluate Py-torch KNN
accuracy_tr = (y_pred_torch == y_test_tensors).float().mean().item()

print(f'Accuarcy (Py-torch KNN Classifer): {accuracy_tr:.2f}')


Accuarcy (Py-torch KNN Classifer): 1.00


In [23]:
h = .02
X_min = X[:, 0].min() - 1 
y_min = X[:, 1].min() - 1 
X_max = X[:, 0].max() + 1
y_max = X[:, 1].max() + 1

xx, yy = np.meshgrid(np.arange(X_min, X_max, h), np.arange(y_min, y_max, h))
xx, yy

(array([[3.3 , 3.32, 3.34, ..., 8.84, 8.86, 8.88],
        [3.3 , 3.32, 3.34, ..., 8.84, 8.86, 8.88],
        [3.3 , 3.32, 3.34, ..., 8.84, 8.86, 8.88],
        ...,
        [3.3 , 3.32, 3.34, ..., 8.84, 8.86, 8.88],
        [3.3 , 3.32, 3.34, ..., 8.84, 8.86, 8.88],
        [3.3 , 3.32, 3.34, ..., 8.84, 8.86, 8.88]], shape=(220, 280)),
 array([[1.  , 1.  , 1.  , ..., 1.  , 1.  , 1.  ],
        [1.02, 1.02, 1.02, ..., 1.02, 1.02, 1.02],
        [1.04, 1.04, 1.04, ..., 1.04, 1.04, 1.04],
        ...,
        [5.34, 5.34, 5.34, ..., 5.34, 5.34, 5.34],
        [5.36, 5.36, 5.36, ..., 5.36, 5.36, 5.36],
        [5.38, 5.38, 5.38, ..., 5.38, 5.38, 5.38]], shape=(220, 280)))

In [24]:
def plot_decision_boundary(X, y, model):
    h = .02  # Step size in the mesh
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    
    # Convert NumPy array to PyTorch tensor before prediction
    Z = model.predict(torch.tensor(np.c_[xx.ravel(), yy.ravel()], dtype=torch.float32))
    Z = Z.numpy().reshape(xx.shape)  # Convert tensor back to NumPy for plotting
    
    plt.contourf(xx, yy, Z, alpha=0.3)
    scatter = plt.scatter(X[:, 0], X[:, 1], c=y, edgecolor='k', cmap=plt.cm.Paired)
    plt.xlabel('Feature 1')
    plt.ylabel('Feature 2')
    plt.title(f'KNN Decision Boundary (k={k})')
    plt.legend(*scatter.legend_elements(), title="Classes")
    plt.show()
